In [ ]:
#SETUP always run this
import math
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
path = "stockcharts.csv"

allData = pd.read_csv(path)

#drop rows where sector is Nan
allData = allData.dropna(subset=['Sector'])

# Create a dictionary of all the sectors mapped to a list of all the industries in that sector mapped to a list of all the tickers in that industry
sectors = {}
for index, row in allData.iterrows():
    if row['Sector'] not in sectors:
        sectors[row['Sector']] = {}
    if row['Industry'] not in sectors[row['Sector']]:
        sectors[row['Sector']][row['Industry']] = []
    sectors[row['Sector']][row['Industry']].append(row['Ticker'])


allSectors = list(sectors.keys())
allIndustries = []
for sector in allSectors:
    allIndustries.extend(list(sectors[sector].keys()))
allIndustries = list(set(allIndustries))
allTickers = []
for sector in allSectors:
    for industry in sectors[sector]:
        allTickers.extend(sectors[sector][industry])
allTickers = list(set(allTickers))




In [ ]:
import json
json.dump(sectors, open("sectors.json",'w'))

In [ ]:
test = json.load(open("sectors.json",'r'))

In [ ]:
sectors

In [ ]:
print(allSectors, len(allSectors))
print(allIndustries, len(allIndustries))
print(allTickers, len(allTickers))

In [ ]:
#download data and save Close price
anchorData = yf.download(allTickers, start = "2020-01-01", end="2023-03-01")
anchorData.head()
closingPrices = anchorData["Close"]

In [ ]:
closingPrices.tail()

In [ ]:
closingPrices.to_csv("BreadthData.csv")

In [ ]:

closingPrices = pd.read_csv("BreadthData.csv")
closingPrices.tail()

In [ ]:
#remove date column from closing prices
closingPrices = closingPrices.drop(columns=["Date"])

In [ ]:

movingAverages = {}
for sector in allSectors:
    print(sector)
    for industry in sectors[sector]:
        count = 0
        total = 0
        for ticker in sectors[sector][industry]:
            movingAverages[(ticker, 20)] = closingPrices[ticker][-20::].mean()
            movingAverages[(ticker, 50)] = closingPrices[ticker][-50::].mean()
            movingAverages[(ticker, 200)] = closingPrices[ticker][-200::].mean()



In [ ]:
MAs = [20,50,200]
counts = {}
for sector in allSectors:
    sectorCount = {}
    sectorTotal = 0
    print(sector)
    for industry in sectors[sector]:
        count = {}
        total = 0
        for ticker in sectors[sector][industry]:
            sectorTotal +=1
            total +=1
            for ma in MAs:
                if closingPrices[ticker].iloc[-1] > movingAverages[(ticker, ma)]:
                    count[ma] = count.get(ma,0)+1
                    sectorCount[ma] = sectorCount.get(ma,0)+1

        percents = {i : count[i]/total for i in count}
        counts[industry] = percents
    sectorPercent = {i : sectorCount[i]/sectorTotal for i in sectorCount}
    counts[sector] = sectorPercent
    


In [ ]:
counts

In [ ]:
#Create a dictionary of sectors mapped to a dictionary of industries mapped to their counts
percents = {}
for sector in allSectors:
    percents[sector] = {}
    percents[sector]["Sector"] = counts[sector]
    for industry in sectors[sector]:
        percents[sector][industry] = counts[industry]
    

percents


In [ ]:
#Create a dictionary of sectors mapped to a list of their industries
industries = {}
for sector in allSectors:
    industries[sector] = list(sectors[sector].keys())

industries